# Wien GTFS shapes to map

The below code vizualises Wien GTFS (General Transit Feed Specification) shapes data on a html map.

## Imports

In [1]:
import os
import sqlite3
import datetime

import folium #https://pypi.org/project/folium/
from pathlib import Path

USER = "xn"
DB_NAME = "wien_gtfs_db.db"
DB_PATH = Path(f"/home/{USER}/Documents/wien_gtfs/db")
MAP_PATH = Path(f"/home/{USER}/Documents/wien_gtfs/map")

## Mapping with folium

In [2]:
%%time

wien_location = [48.210033, 16.363449]

mapit = folium.Map(crs='EPSG3857', 
                   location=wien_location, 
                   tiles='Cartodbdark_matter', # None 'stamentoner'
                   zoom_start=12)

con = sqlite3.connect(DB_PATH/DB_NAME)
cursor = con.cursor()

cursor.execute("SELECT DISTINCT shape_id FROM shapes")
distinct_shapes = cursor.fetchall()

for shape in distinct_shapes:
    cursor.execute(f"SELECT * FROM shapes WHERE shape_id = '{shape[0]}'")
    one_shape = cursor.fetchall()
    coordinate_list = []
    for line in one_shape:
        shape_id = line[0]
        coorinate = (line[1],line[2])
        coordinate_list.append(coorinate)
        
    folium.PolyLine(locations=coordinate_list, 
                    tooltip=shape_id,
                    color="red", 
                    weight=2, 
                    opacity=0.75).add_to(mapit)

cursor.close()
con.close()

mapit.save(MAP_PATH/"wien_lines.html")

CPU times: user 9.64 s, sys: 1.86 s, total: 11.5 s
Wall time: 11.5 s
